In [1]:
!pip install datasets -q

In [11]:
import ast

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.base import BaseEstimator, TransformerMixin

from datasets import load_dataset

import torch

import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataset = load_dataset("BAAI/TACO")

In [ ]:
sample = next(iter(dataset['train']))

In [ ]:
print(f'Keys: {list(sample.keys())}')
print(f"Count of Keys: {len(sample.keys())}")

In [ ]:
print(sample['question'])

In [ ]:
sample['solutions']

In [ ]:
trainData = dataset['train'].to_pandas()
testData = dataset['test'].to_pandas()

In [ ]:
trainData.head(4)

In [ ]:
type(trainData['solutions'][0]), type(testData['solutions'][0])

In [ ]:
trainData['solutions'] = trainData['solutions'].apply(lambda x: ast.literal_eval(x))
testData['solutions'] = testData['solutions'].apply(lambda x: ast.literal_eval(x))

In [ ]:
type(trainData['solutions'][0]), type(testData['solutions'][0])

In [ ]:
sns.displot(trainData['difficulty'])
plt.xticks(rotation=60)
plt.show()

In [ ]:
sns.displot(trainData['difficulty'])
plt.xticks(rotation=60)
plt.show()

In [ ]:
sns.displot(trainData['source'])
plt.xticks(rotation=60)
plt.show()

In [ ]:
sample = trainData.sample(10)

In [ ]:
class Preprocess(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def chooseOneSolution(self, df):
        df = df[df['solutions'].apply(len) > 0]
        
        def choose_random(arr):
            return np.random.choice(arr)

        df['solutions'] = df['solutions'].apply(choose_random)
        return df
    
    def fit(self, df):
        df = df[['question', 'solutions']]
        df = self.chooseOneSolution(df)
        return df
    

preprocessClass = Preprocess()

In [ ]:
preprocessClass.fit(sample)

In [ ]:
trainData = preprocessClass.fit(trainData)
testData = preprocessClass.fit(testData)

In [ ]:
len(trainData), len(testData)

In [ ]:
trainData.to_csv('Train.csv')
testData.to_csv('Test.csv')

In [3]:
trainData = pd.read_csv('/kaggle/input/text-to-code-generation/Train.csv')
testData = pd.read_csv('/kaggle/input/text-to-code-generation/Test.csv')

In [4]:
trainData.head(2)

,Unnamed: 0,question,solutions
0,1,There are $n$ candy boxes in front of Tania. T...,"(n, s, k) = map(int, input().split())\nr = lis..."
1,3,"If you visit Aizu Akabeko shrine, you will fin...",n = input()\nlength = len(n)\nans = 10\nlst = ...


In [7]:
print(trainData.iloc[1]['solutions'])

n = input()
length = len(n)
ans = 10
lst = []
ind = 0
while ind < length:
	if n[ind] == '1' and ind + 1 <= length - 1:
		lst.append(int(n[ind:ind + 2]))
		ind += 2
	else:
		lst.append(int(n[ind]))
		ind += 1
if len(lst) >= 2:
	ans = min(ans, max(lst) - min(lst))
divisors = []
for i in range(1, length // 2 + 1):
	if length % i == 0:
		divisors.append(i)
for i in divisors:
	lst = []
	for j in range(0, length, i):
		lst.append(int(n[j:j + i]))
	ans = min(ans, max(lst) - min(lst))
print(ans)



In [19]:
from tokenize import tokenize, untokenize
import io


def tokenize_python_code(python_code_str):
    python_tokens = list(tokenize(io.BytesIO(python_code_str.encode('utf-8')).readline))
    tokenized_output = []
    for i in range(0, len(python_tokens)):
        tokenized_output.append((python_tokens[i].type, python_tokens[i].string))
    return tokenized_output

In [28]:
print(trainData['solutions'][0])

(n, s, k) = map(int, input().split())
r = list(map(int, input().split()))
s -= 1
c = input()
best = [[0 for i in range(n)] for j in range(k + 1)]
for i in range(1, k + 1):
	for j in range(n):
		if i <= r[j]:
			best[i][j] = abs(j - s)
		else:
			good = float('inf')
			for l in range(n):
				if c[j] != c[l] and r[j] > r[l]:
					good = min(good, best[i - r[j]][l] + abs(j - l))
			best[i][j] = good
if min(best[-1]) == float('inf'):
	print(-1)
else:
	print(min(best[-1]))



In [27]:
len(tokenize_python_code(trainData['solutions'][0])), print(untokenize(tokenize_python_code(trainData['solutions'][0])))

b"(n ,s ,k )=map (int ,input ().split ())\nr =list (map (int ,input ().split ()))\ns -=1 \nc =input ()\nbest =[[0 for i in range (n )]for j in range (k +1 )]\nfor i in range (1 ,k +1 ):\n\tfor j in range (n ):\n\t\tif i <=r [j ]:\n\t\t\tbest [i ][j ]=abs (j -s )\n\t\telse :\n\t\t\tgood =float ('inf')\n\t\t\tfor l in range (n ):\n\t\t\t\tif c [j ]!=c [l ]and r [j ]>r [l ]:\n\t\t\t\t\tgood =min (good ,best [i -r [j ]][l ]+abs (j -l ))\n\t\t\tbest [i ][j ]=good \nif min (best [-1 ])==float ('inf'):\n\tprint (-1 )\nelse :\n\tprint (min (best [-1 ]))\n"


(252, None)